In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib

In [3]:
%matplotlib widget

In [4]:
from overfit.trainers.overfit import OverfitTrainer
import matplotlib.pyplot as plt
from PIL import Image
from overfit.utils.img2vid import zigzag, display_video
import torchvision.transforms.functional as FT
from torchvision.models import resnet34, ResNet34_Weights
from torchvision.models import resnet50, ResNet50_Weights
import urllib
import mlflow

In [5]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [6]:
input_image = Image.open("dog.jpg")
input_tensor = FT.to_tensor(input_image)
_, h, w = input_tensor.size()
input_video = zigzag(input_tensor, h // 3, w // 3)
# vid = display_video(input_video)
# vid.save("dog.mp4")
print(len(input_video))

69


In [7]:
def normalize_rgb(img):
    return FT.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
input_video = [normalize_rgb(frame).unsqueeze(0) for frame in input_video]

In [8]:
srcnet = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1).eval()
tgtnet_trainer = OverfitTrainer()
tgtnet_trainer.set(
    pretrained_classifier=srcnet,
    num_classes=1000,
    confidence=0.1,
    weight_decay=1,
    max_lr=0.1,
    momentum=0.9,
)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /Users/dgcnz/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [9]:
tgtnet_trainer.new_experiment()
tgtnet_trainer.test(input_video, [258] * len(input_video))

/Users/dgcnz/development/research/overfitting-net/.venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
